In [ ]:
#  jupyter notebook --ip=192.168.2.118 --no-browser --port=8080 --allow-root

In [1]:
from scapy.all import *
from scapy.layers.tls.all import *
import os

In [2]:
print(scapy.__version__)

2.5.0


In [6]:
# load_layer('tls')

# conf.tls_session_enable = True
# conf.tls_nss_filename = 'SSLKEYLOGFILE'

In [7]:
pcap = rdpcap("ssl_test.pcap")

In [8]:
pcap[0].show()

###[ Ethernet ]### 
  dst       = 44:6d:57:4b:ed:0b
  src       = 00:80:48:49:11:64
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 203
     id        = 18185
     flags     = DF
     frag      = 0
     ttl       = 128
     proto     = tcp
     chksum    = 0x2fcc
     src       = 192.168.1.4
     dst       = 192.168.1.3
     \options   \
###[ TCP ]### 
        sport     = 2061
        dport     = https
        seq       = 2908474314
        ack       = 422420580
        dataofs   = 5
        reserved  = 0
        flags     = PA
        window    = 10164
        chksum    = 0x1a6b
        urgptr    = 0
        options   = []
###[ TLS ]### 
           type      = handshake
           version   = TLS 1.0
           len       = 158    [deciphered_len= 158]
           iv        = b''
           \msg       \
            |###[ TLS Handshake - Client Hello ]### 
            |  msgtype   = client_hello
            |  msglen    = 154
 

## Edit SNI value to custom value

In [11]:
pcap_file = "ssl_test.pcap"
new_sni = 'fieabjfafitcaid.com'

def rectifySeqAckNo(packets):
    srcIp = packets[0]["IP"].src
    dstIp = packets[0]["IP"].dst
    seqNo = 0
    ackNo = 0
    currDir = 0
    prevDir = 0
    for packet in packets:
        if(packet["IP"].src == srcIp):
            currDir = 0
        else:
            currDir = 1
        if(currDir != prevDir):
            seqNo,ackNo = ackNo,seqNo
        packet["TCP"].seq = seqNo
        packet["TCP"].ack = ackNo
        seqNo+=len(packet["TCP"].payload)
        prevDir = currDir
        del packet['TCP'].chksum

def chnageSniValue(pcap_file, new_sni):
    packets = rdpcap(pcap_file)  # Assuming there's only one packet in the pcap file
    
    packet = packets[0]
    # Check if the packet contains TLS layer
    if packet.haslayer(TLS):
        # Find SNI extension
        sni_extension = packet['TLS']['TLS_Ext_ServerName']
        len_change = 0
        if sni_extension:
            # Modify SNI server name
            old_sni = sni_extension.servernames[0].servername
            old_sni_len = sni_extension.servernames[0].namelen
            len_change = len(new_sni)-old_sni_len
            print ("length change = ",len_change)
            sni_extension.servernames[0].servername = new_sni
    
            del sni_extension.servernames[0].namelen
            del packet['TLS']["TLS Handshake - Client Hello"]["TLS Extension - Server Name"].len
            del packet['TLS']["TLS Handshake - Client Hello"]["TLS Extension - Server Name"].servernameslen
            del packet['TLS']["TLS Handshake - Client Hello"].extlen
            del packet['TLS']["TLS Handshake - Client Hello"].msglen
            del packet['TLS'].len
            del packet['IP'].len
    
            rectifySeqAckNo(packets)
            
            # Write the modified packet to a new pcap file
            new_pcap_file = pcap_file.replace('.pcap', '-customSni.pcap')
            wrpcap(new_pcap_file, packets)
            os.chmod(new_pcap_file, 0o777)
            print(f"Modified pcap file saved as: {new_pcap_file}")

chnageSniValue(pcap_file, new_sni)

length change =  9
Modified pcap file saved as: ssl_test-customSni.pcap


## Add multiple SNI value

In [10]:
pcap_file = "ssl_test.pcap"
serverNameStr = 'multiplesni.com'

def rectifySeqAckNo(packets):
    srcIp = packets[0]["IP"].src
    dstIp = packets[0]["IP"].dst
    seqNo = 0
    ackNo = 0
    currDir = 0
    prevDir = 0
    for packet in packets:
        if(packet["IP"].src == srcIp):
            currDir = 0
        else:
            currDir = 1
        if(currDir != prevDir):
            seqNo,ackNo = ackNo,seqNo
        packet["TCP"].seq = seqNo
        packet["TCP"].ack = ackNo
        seqNo+=len(packet["TCP"].payload)
        prevDir = currDir
        del packet['TCP'].chksum
        
def multipleSniValue(pcap_file, new_sni):
    packets = rdpcap(pcap_file)  # Assuming there's only one packet in the pcap file
    packet = packets[0]
    # Check if the packet contains TLS layer
    if packet.haslayer(TLS):
        # Find SNI extension
        sni_extension = packet['TLS']['TLS_Ext_ServerName']
        len_change = 0
        if sni_extension:
            # Modify SNI server name
            old_sni = sni_extension.servernames[0].servername
            old_sni_len = sni_extension.servernames[0].namelen
            len_change = len(new_sni)-old_sni_len
            print ("length change = ",len_change)
            # sni_extension.servernames[0].servername = new_sni

            # Append SNI
            servernameLen = len(serverNameStr)
            # Convert integer to bytes
            byte_literal = servernameLen.to_bytes(2, byteorder='big')
            packets[0]['TLS']["TLS Handshake - Client Hello"]["TLS Extension - Server Name"].servernames.append(ServerName(b"\x00"+byte_literal+serverNameStr.encode('utf-8')))
    
            del sni_extension.servernames[0].namelen
            del packet['TLS']["TLS Handshake - Client Hello"]["TLS Extension - Server Name"].len
            del packet['TLS']["TLS Handshake - Client Hello"]["TLS Extension - Server Name"].servernameslen
            del packet['TLS']["TLS Handshake - Client Hello"].extlen
            del packet['TLS']["TLS Handshake - Client Hello"].msglen
            del packet['TLS'].len
            del packet['IP'].len

            rectifySeqAckNo(packets)
            # Write the modified packet to a new pcap file
            new_pcap_file = pcap_file.replace('.pcap', '-multipleSni.pcap')
            wrpcap(new_pcap_file, packets)
            os.chmod(new_pcap_file, 0o777)
            print(f"Modified pcap file saved as: {new_pcap_file}")
            # packets[0].show()
multipleSniValue(pcap_file, serverNameStr)

length change =  5
Modified pcap file saved as: ssl_test-multipleSni.pcap
###[ Ethernet ]### 
  dst       = 44:6d:57:4b:ed:0b
  src       = 00:80:48:49:11:64
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = None
     id        = 18185
     flags     = DF
     frag      = 0
     ttl       = 128
     proto     = tcp
     chksum    = 0x2fcc
     src       = 192.168.1.4
     dst       = 192.168.1.3
     \options   \
###[ TCP ]### 
        sport     = 2061
        dport     = https
        seq       = 0
        ack       = 0
        dataofs   = 5
        reserved  = 0
        flags     = PA
        window    = 10164
        chksum    = None
        urgptr    = 0
        options   = []
###[ TLS ]### 
           type      = handshake
           version   = TLS 1.0
           len       = None    [deciphered_len= 158]
           iv        = b''
           \msg       \
            |###[ TLS Handshake - Client Hello ]### 
            |  

In [9]:
def rectifySeqAckNo(packets):
    srcIp = packets[0]["IP"].src
    dstIp = packets[0]["IP"].dst
    seqNo = 0
    ackNo = 0
    currDir = 0
    prevDir = 0
    for packet in packets:
        if(packet["IP"].src == srcIp):
            currDir = 0
        else:
            currDir = 1
        if(currDir != prevDir):
            seqNo,ackNo = ackNo,seqNo
        packet["TCP"].seq = seqNo
        packet["TCP"].ack = ackNo
        seqNo+=len(packet["TCP"].payload)
        prevDir = currDir
        del packet['TCP'].chksum
        # print(str(packet["TCP"].ack)+"-"+str(packet["TCP"].seq)+"="+str(len(packet["TCP"].payload)))


packets = rdpcap("ssl_test.pcap")
rectifySeqAckNo(packets)

0-0=163
163-0=830
830-163=186
349-830=238
1068-349=27
1068-376=163
539-1068=830
1898-539=186
725-1898=238
2136-725=27
2136-752=163
915-2136=830
2966-915=186
1101-2966=238
3204-1101=310
1411-3204=273
3477-1411=27
3477-1438=371
1809-3477=133
3610-1809=338
2147-3610=307
3917-2147=27
3917-2174=371
2545-3917=133
4050-2545=368
2913-4050=307
4357-2913=27
4357-2940=371
3311-4357=133
4490-3311=368
3679-4490=1460
3679-5950=1460
3679-7410=1460
3679-8870=1460
3679-10330=1460
3679-11790=1460
3679-13250=1460
3679-14710=1283
15993-3679=27
15993-3706=371
4077-15993=133
16126-4077=511
4588-16126=593
16719-4588=27
16719-4615=371
4986-16719=133
16852-4986=787
5773-16852=1135
17987-5773=27


In [68]:
integer_value = 20
num_bytes = 2  # Number of bytes to represent the integer

# Convert integer to bytes
byte_literal = integer_value.to_bytes(num_bytes, byteorder='big')

print(byte_literal)

b'\x00\x14'


In [62]:
pcap[0]['TLS']["TLS Handshake - Client Hello"]["TLS Extension - Server Name"].servernames.append(snicpy)

In [63]:
pcap[0]['TLS']["TLS Handshake - Client Hello"]["TLS Extension - Server Name"].servernames

[<ServerName  nametype=host_name namelen=10 servername='bawmashbij' |>,
 <ServerName  nametype=host_name namelen=10 servername='bawmashbij' |>]

In [16]:
packet['TLS']["TLS Handshake - Client Hello"]["TLS Extension - Server Name"].len+=len_change
packet['TLS']["TLS Handshake - Client Hello"]["TLS Extension - Server Name"].servernameslen+=len_change
packet['TLS']["TLS Handshake - Client Hello"]["TLS Extension - Server Name"]

<TLS_Ext_ServerName  type=server_name len=24 servernameslen=22 servernames=[b'fieabjfafitcaid.com'] |>

In [14]:
len_change

9

In [ ]:
<TLS_Ext_ServerName  type=server_name len=15 servernameslen=13 servernames=[b'fieabjfafitcaid.com'] |>